In [1]:
import numpy as np

import torch
from torch import nn
from torch import functional as F

from livelossplot import PlotLosses

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
from Inference import BBVI 

In [3]:
from Inference.BBVI import VariationalNetwork

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
sigma_noise = 0.1
data = torch.load('Data/foong_data.pt')
x_data = data[0].to(device)
y_data = data[1].to(device)
y_data = y_data.unsqueeze(-1)

In [62]:
class MixtureVariationalNetwork(nn.Module):
    def __init__(self, input_size, output_size, layer_width, nb_layers, device=None):
        super(MixtureVariationalNetwork, self).__init__()
        
        self.layer_width = layer_width
        self.input_size = input_size
        self.output_size = output_size
        self.nb_layers = nb_layers
        self.device = device
        
        self.components = []
        self.pi = torch.tensor([], device=device)
        
    def add_component(self, component, proportion):
        #todo check compatibility with other components
        component.requires_grad_rhos(False)
        component.requires_grad_mus(False)
        self.components.append(component)
        proportion = proportion.to(self.device)
        self.pi = torch.cat((self.pi*(1-proportion), proportion.unsqueeze(0)))
        self.pi.require_grad = False
        
    def sample_parameters(self, M=1, new_component=None, new_proportion=torch.tensor(0.0)):
        npi = torch.tensor([1.0-new_proportion, new_proportion], device=self.device)
        D = torch.distributions.multinomial.Multinomial(M, npi)
        m_oldnew = D.sample()
        if int(m_oldnew[0]) > 0:
            D_old = torch.distributions.multinomial.Multinomial(int(m_oldnew[0]), self.pi)
            m_old = D_old.sample()
        else:
            m_old = torch.zeros(self.pi.size())

        S = []    
        for j in range(len(self.pi)):
            t = self.components[j].sample_parameters(int(m_old[j]))
            if int(m_old[j]) > 0: 
                S.append(t)
        if new_component is not None:
            S.append(new_component.sample_parameters(int(m_oldnew[1])))

        return ([torch.cat([c[0][k] for c in S]) for k in range(self.nb_layers)], [torch.cat([c[1][k] for c in S]) for k in range(self.nb_layers)])

    def q_log_pdf(self, layered_w_samples, layered_bias_samples):
        log_q = [c.q_log_pdf(layered_w_samples, layered_bias_samples) for c in self.components]
        return torch.logsumexp(torch.stack(log_q) + torch.log(self.pi).unsqueeze(0).t(), dim=0)

    def prior_log_pdf(self, layered_w_samples, layered_bias_samples):
        log_prior = [c.prior_log_pdf(layered_w_samples, layered_bias_samples) for c in self.components]
        return torch.logsumexp(torch.stack(log_prior) + torch.log(self.pi).unsqueeze(0).t(), dim=0)
    
    def full_elbo(self, x_data, y_data, n_samples_ELBO, sigma_noise, new_component=None, new_proportion=None):
        (layered_w_samples_XC, layered_bias_samples_XC) = self.sample_parameters(n_samples_ELBO, new_component, new_unscaled_proportion)

        logprior_XC = self.prior_log_pdf(layered_w_samples_XC, layered_bias_samples_XC)

        y_pred_XC = self.forward(x_data)
        loglikelihood_XC = self._log_norm(y_pred_XC, y_data, torch.tensor(sigma_noise).to(self.device))

        logposterior_XC = (logprior_XC.unsqueeze(-1).unsqueeze(-1).repeat([1,x_data.shape[0],1]) + loglikelihood_XC).sum(dim=[1,2])

        logq_XC = self.q_log_pdf(layered_w_samples_XC, layered_bias_samples_XC)

        logqnew_XC = new_component.q_log_pdf(layered_w_samples_XC, layered_bias_samples_XC)

        logqall_XC = torch.exp(torch.logsumexp( \
        torch.stack([torch.log(torch.tensor(1.0, device=self.device)-new_proportion) + logq_XC, \
                    torch.log(new_proportion) + logqnew_XC], dim=0), dim=0))

        return (logposterior_XC - logqall_XC).sum()
        
    def compute_elbo(self, x_data, y_data, n_samples_ELBO, sigma_noise, new_component=None, new_proportion=None):
        # sample X^(c)
        (layered_w_samples_XC, layered_bias_samples_XC) = self.sample_parameters(n_samples_ELBO)

        LP_XC = self.prior_log_pdf(layered_w_samples_XC, layered_bias_samples_XC)
        y_pred_XC = self.forward(x_data)
        LL_XC = self._log_norm(y_pred_XC, y_data, torch.tensor(sigma_noise).to(self.device))
        posterior_XC = torch.sum(LP_XC.unsqueeze(-1).unsqueeze(-1) + LL_XC, dim=[1,2])
        posterior_XC = (LP_XC.unsqueeze(-1).unsqueeze(-1).repeat([1,x_data.shape[0],1]) + LL_XC).sum(dim=[1,2])
        
        qC_log_XC = self.q_log_pdf(layered_w_samples_XC, layered_bias_samples_XC)
        
        if new_component is None:
            return torch.mean(qC_log_XC - posterior_XC)
        
        p = torch.sigmoid(new_proportion)
        
        qN_log_XC = new_component.q_log_pdf(layered_w_samples_XC, layered_bias_samples_XC)
        qCN_log_XC = torch.logsumexp(torch.stack([torch.log(torch.tensor(1.0, device=self.device)-p) + qC_log_XC, torch.log(p) + qN_log_XC],dim=0),dim=0)
        
        # sample X_(c+1)
        (layered_w_samples_XN, layered_bias_samples_XN) = new_component.sample_parameters(n_samples_ELBO)
        
        LP_XN = new_component.prior_log_pdf(layered_w_samples_XN, layered_bias_samples_XN)
        y_pred_XN = new_component.forward(x_data)
        LL_XN = new_component._log_norm(y_pred_XN, y_data, torch.tensor(sigma_noise).to(self.device))

        posterior_XN = (LP_XN.unsqueeze(-1).unsqueeze(-1).repeat([1,x_data.shape[0],1]) + LL_XN).sum(dim=[1,2])
        
        qC_log_XN = self.q_log_pdf(layered_w_samples_XN, layered_bias_samples_XN)
        
        qN_log_XN = new_component.q_log_pdf(layered_w_samples_XN, layered_bias_samples_XN)
        qCN_log_XN = torch.logsumexp(torch.stack([torch.log(torch.tensor(1.0, device=self.device)-p) + qC_log_XN, torch.log(p) + qN_log_XN],dim=0),dim=0)
        
        L = (torch.tensor(1.0, device=self.device)-p) * (posterior_XC.mean() - qCN_log_XC.mean()) + p * (posterior_XN.mean() - qCN_log_XN.mean()) 
        
        return -L
    
    def requires_grad_rhos(self, v = False):
        for k in range(len(self.components)):
            self.components[k].requires_grad_rhos(v)
        
    def requires_grad_mus(self, v = False):
        for k in range(len(self.components)):
            self.components[k].requires_grad_mus(v)
    
    def forward(self, x):
        return torch.cat([self.components[k].forward(x_data) for k in range(len(self.components))],dim=0)
    

    def _log_norm(self, x, mu, std):
        return -0.5 * torch.log(2*np.pi*std**2) -(0.5 * (1/(std**2))* (x-mu)**2)

In [63]:
class VariationalBoostingOptimizer():
    def __init__(self, mixture, sigma_noise, optimizer, optimizer_params, scheduler=None, scheduler_params=None):
        self.mixture = mixture
        self.sigma_noise = sigma_noise
        self.device = model.device
        
        self.optimizer = optimizer
        if scheduler is None:           
            self.scheduler = None
        else:
            self.scheduler = scheduler
            
    def run(self, data, n_epoch=100, n_iter=1, n_ELBO_samples=1, seed=None, plot=False, savePath=None, xpName=None, networkName=None, saveName=None):
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)

        x_data = data[0].to(self.device)
        y_data = data[1].to(self.device)

        self.optimizer.zero_grad()

        if saveName is not None and savePath is not None:
            liveloss = PlotLosses(fig_path=str(savePath)+str(xpName)+'_'+str(networkName)+'_'+str(saveName))
        else:
            liveloss = PlotLosses()

        for j in range(n_epoch):
            logs = {}
            losses = [None] * n_iter

            for k in range(n_iter):
                self.optimizer.zero_grad()
                loss = self.model.compute_elbo(x_data, y_data, n_ELBO_samples, self.sigma_noise, self.device)
                losses[k] = loss
                loss.backward()
                self.optimizer.step()

            logs['expected_loss'] = torch.stack(losses).mean().detach().clone().cpu().numpy()
            logs['learning rate'] = self.optimizer.param_groups[0]['lr']
            liveloss.update(logs)
            if plot is True:
                liveloss.draw()
            if self.scheduler is not None:
                self.scheduler.step(logs['expected_loss'])
        return self.model

In [64]:
c1 = VariationalNetwork(1, 1, 20, 2, device=device)

In [65]:
mix = MixtureVariationalNetwork(1, 1, 20, 2, device=device)

In [66]:
mix.add_component(c1, torch.tensor(1.0))

In [67]:
new_component = VariationalNetwork(1, 1, 20, 2, device=device)

In [68]:
new_unscaled_proportion = nn.Parameter(torch.tensor(0.0, requires_grad=True, device=device))
new_proportion = torch.sigmoid(new_unscaled_proportion)

In [69]:
n_samples_ELBO = 1000

In [70]:
mix.compute_elbo(x_data, y_data, n_samples_ELBO, sigma_noise, new_component=new_component, new_proportion=new_proportion)

tensor(4263772.5000, device='cuda:0', grad_fn=<NegBackward>)

In [71]:
mix.full_elbo(x_data, y_data, n_samples_ELBO, sigma_noise, new_component=new_component, new_proportion=new_proportion)

tensor(-9.3136e+08, device='cuda:0', grad_fn=<SumBackward0>)